In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Constants
INPUT_DATA_PATH = "/kaggle/input/ashrae-energy-prediction"

In [ ]:
# Load raw data
train_df = pd.read_csv(f"{INPUT_DATA_PATH}/train.csv")
building_df = pd.read_csv(f"{INPUT_DATA_PATH}/building_metadata.csv")
weather_train_df = pd.read_csv(f"{INPUT_DATA_PATH}/weather_train.csv")

In [ ]:
train_df["building_id"] = train_df["building_id"].astype("category")
train_df["meter"] = train_df["meter"].astype("category")
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
train_df["meter_reading"] = train_df["meter_reading"].astype(float)

In [ ]:
building_df["site_id"] = building_df["site_id"].astype("category")
building_df["building_id"] = building_df["building_id"].astype("category")

In [ ]:
weather_train_df["site_id"] = weather_train_df["site_id"].astype("category")
weather_train_df["timestamp"] = pd.to_datetime(weather_train_df["timestamp"])

In [ ]:
# Merge train and buildings df
merged_train_df = pd.merge(
    left=train_df,
    right=building_df,
    on="building_id",
    how="left",
)

In [ ]:
merged_train_df = merged_train_df.drop(columns="_merge")

In [ ]:
merged_train_df.head()

In [ ]:
merged_train_df = pd.merge(
    left=merged_train_df,
    right=weather_train_df,
    on=["site_id", "timestamp"],
    how="left",
    indicator=True
)

In [ ]:
merged_train_df[(merged_train_df["_merge"] != "both")]

In [ ]:
weather_test = pd.read_csv(f"{INPUT_DATA_PATH}/weather_test.csv")

In [ ]:
weather_